<p style="text-align: center;font-size:50px;">M. Ammar Siddique <br> BSDSF22M041</p>


<p style="text-align: center;font-size:50px;">Data Mining and Machine Learning </p>


<p style="text-align: center;font-size:50px;">Assignment # 5 </p>

                                                         
<p style="text-align: center;font-size:40px;">Global Water Consumption Data Set </p>

## Iterative Dichotomiser (ID-3)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Load the dataset
df = pd.read_csv("global_water_consumption.csv")

# 2. Drop missing values (or use imputation if necessary)
df.dropna(inplace=True)

# 3. Select target variable
target_column = "Total Water Consumption (Billion Cubic Meters)"
y = df[target_column]
X = df.drop(columns=[target_column, "Country", "Year"])  # Drop non-numeric features

# 4. Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(exclude=["object"]).columns

# 5. Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

# 6. Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 7. Create a Decision Tree pipeline
dt_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", DecisionTreeRegressor(random_state=42))
])

# 8. Define hyperparameter grid for tuning
param_grid = {
    "regressor__max_depth": [3, 5, 10, None],
    "regressor__min_samples_split": [2, 5, 10],
    "regressor__min_samples_leaf": [1, 2, 4]
}

# 9. Perform Grid Search for best parameters
dt_search = GridSearchCV(dt_pipeline, param_grid, cv=5, scoring="r2")
dt_search.fit(X_train, y_train)

# 10. Make Predictions
y_pred = dt_search.best_estimator_.predict(X_test)

# 11. Evaluate Model Performance
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("=== Decision Tree Regression Performance ===")
print(f"MSE  : {mse}")
print(f"MAE  : {mae}")
print(f"R²   : {r2}")

# 12. Feature Importance
importances = dt_search.best_estimator_.named_steps["regressor"].feature_importances_
feature_names = numerical_cols.tolist() + list(
    dt_search.best_estimator_.named_steps["preprocessor"].named_transformers_["cat"]
    .get_feature_names_out(categorical_cols)
)
plt.figure(figsize=(8, 5))
sns.barplot(x=importances, y=feature_names)
plt.title("Feature Importance in Decision Tree")
plt.show()

# 13. Actual vs Predicted Scatter Plot
plt.figure(figsize=(7, 5))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red")  # Diagonal Line
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Decision Tree Regression: Actual vs Predicted")
plt.show()

# 14. Residual Plot
residuals = y_test - y_pred
plt.figure(figsize=(7, 5))
sns.histplot(residuals, bins=30, kde=True)
plt.axvline(0, color="red", linestyle="dashed")
plt.xlabel("Residuals")
plt.title("Residual Distribution")
plt.show()
